In [2]:
import tensorflow as tf
from tensorflow.python.tools import freeze_graph
from tensorflow.python.tools import optimize_for_inference_lib


from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD,RMSprop,adam
from keras.utils import np_utils

import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import os
import theano
from PIL import Image
from numpy import *
# SKLEARN
from sklearn.utils import shuffle
from sklearn.cross_validation import train_test_split


img_rows , img_cols = 128 , 128

path1= 'C:\\Users\\User\\Desktop\\PycharmProjects\\images'
path2= 'C:\\Users\\User\\Desktop\\PycharmProjects\\images-resized'

listing = os.listdir(path1)

num_samples =size(listing)
print(num_samples)

for file in listing:
    im= Image.open(path1 + '\\' + file)
    img = im.resize ((img_rows,img_cols))
    gray = img.convert('L')
    
    gray.save(path2 + '\\' + file, "JPEG" )

imlist= os.listdir(path2)
im1 =array(Image.open('C:\\Users\\User\\Desktop\\PycharmProjects\\images-resized' + '\\' + imlist[0]))



Using TensorFlow backend.
C:\Users\User\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


91


In [4]:
immatrix=array ([array(Image.open('C:\\Users\\User\\Desktop\\PycharmProjects\\images-resized' + '\\' + im2)).flatten() 
                    for im2 in imlist] ,'f')
       
m,n = immatrix.shape[0:2]

label = np.ones((num_samples,), dtype = int64 )
label[0:10]=0
label[11:40]=1
label[41:70]=2
label[71:90]=3
print(label)  


[0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 1]


In [5]:
# Number of colour channels for the images: 1 channel for gray-scale.
#https://en.wikipedia.org/wiki/Channel_(digital_image)
#channels mean number of primary colors
data,Label = shuffle(immatrix,label,random_state=2)
train_data = [data,Label]


plt.imshow(immatrix[16].reshape(img_rows,img_cols))
plt.imshow(immatrix[16].reshape(img_rows,img_cols),cmap='gray')
print (train_data[0].shape)
print (train_data[1].shape)


(91, 16384)
(91,)


In [7]:

# Convolutional Layer 1.
#When dealing with high-dimensional inputs such as images, 
#it is impractical to connect neurons to all neurons in the previous volume. 
#Instead, we will connect each neuron to only a local region of the input volume. 
#The spatial extent of this connectivity is a hyperparameter called the receptive field 
#of the neuron (equivalently this is the filter size). 
#smaller size than input

#batch_size to train
batch_size = 64
# number of output classes
nb_classes = 4
# number of epochs to train
nb_epoch = 20


# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
nb_pool = 2
# convolution filter size
nb_conv = 3

(X,y) =(train_data[0],train_data[1])



#split X ,y into training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)
#

X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= 255
X_test /= 255

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

i = 16
plt.imshow(X_train[i, 0], interpolation='nearest')
print("label : ", Y_train[i,:])
print(train_data[0])

X_train shape: (72, 1, 128, 128)
72 train samples
19 test samples
label :  [ 0.  1.  0.  0.]
[[ 192.  193.  191. ...,  213.  213.  215.]
 [ 197.  197.  197. ...,  190.  190.  190.]
 [ 255.  255.  255. ...,  254.  254.  254.]
 ..., 
 [ 200.  198.  199. ...,  194.  194.  194.]
 [ 201.  201.  201. ...,  194.  194.  194.]
 [ 254.  254.  254. ...,  255.  255.  255.]]


In [ ]:
##preprocessing
# 

train_generator = datagen.flow_from_directory(X_train,
                                             target_size=(128,128),
                                             batch_size=batch_size,
                                             class_mode=Y_train)

validation_generator = datagen.flow_from_directory(X_test,
                                                  target_size=())

In [7]:

input_shape=[128,128,1]
					
model = Sequential()

model.add(Convolution2D(32, 3,3,border_mode='same',input_shape=input_shape,activation='relu'))
model.add(Convolution2D(32, 3, 3,activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Convolution2D(64, 3, 3,activation='relu'))
#model.add(Convolution2D(64, 3, 3))
#model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(64,activation='relu'))

model.add(Dropout(0.5))
model.add(Dense(nb_classes,activation='softmax'))

#sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
#model.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=["accuracy"])
model.compile(loss='categorical_crossentropy', optimizer='rmsprop',metrics=["accuracy"])


C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=[128, 128,..., activation="relu", padding="same")`
  
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
  import sys
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  # This is added back by InteractiveShellApp.init_path()


In [8]:


def train(model, X_train, Y_train, X_test, Y_test):
   
    model.fit(X_train, Y_train, \
              batch_size=batch_size, \
              epochs=nb_epoch, \
              verbose=1, \
              validation_data=(X_test, Y_test))
